In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)
import seaborn as sns
import os
import missingno as msno
import pickle
from glob import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

## Import Data

In [ ]:
order_files = glob(f'{os.path.abspath("1. 이커머스 FC주문 데이터")}/1-*')
delivery_files = glob(f'{os.path.abspath("2. 택배 배송 데이터")}/2-*.xlsx')

In [ ]:
order_raw = []
for file in tqdm(order_files):
    try:
        order = pd.read_excel(file, sheet_name=0)
        order_raw.append(order)
    except:
        order = pd.read_csv(file)
        order_raw.append(order)
orders = pd.concat(order_raw)

In [ ]:
delivery_raw = []
for file in tqdm(delivery_files):
    delivery = pd.read_excel(file, index_col=0)
    delivery_raw.append(delivery)
deliveries = pd.concat(delivery_raw)

In [ ]:
orders.to_csv('orders.csv', index=False)
deliveries.to_csv('deliveries.csv', index=False)

## Preprocessing

In [ ]:
orders.BKG_DATE = pd.to_datetime(orders.BKG_DATE, format='%Y%m%d')
orders.INS_DATE = pd.to_datetime(orders.INS_DATE, format='%Y%m%d')
orders["BKG_TIME"] = pd.to_datetime(orders["BKG_TIME"].apply(lambda x : str(x).zfill(6)), format='%H%M%S')

In [ ]:
# 주문 날짜 분류
orders['BKG_MONTH'] = orders.BKG_DATE.dt.month
orders['BKG_DAY'] = orders.BKG_DATE.dt.day
orders['BKG_DAYNAME'] = orders.BKG_DATE.dt.day_name()

orders['BKG_HOUR'] = orders['BKG_TIME'].dt.hour

In [ ]:
orders['BKG_HOUR'] = orders['BKG_TIME'].dt.hour

In [ ]:
# 송화인 주소 전처리
orders.loc[orders.SHPR_ADDR_1=='서울시','SHPR_ADDR_1'] = '서울특별시'
orders.loc[orders.SHPR_ADDR_1=='경기','SHPR_ADDR_1'] = '경기도'

In [ ]:
# 수화인 주소 전처리
orders.loc[orders.CNEE_ADDR_1=='전남목포시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['전라남도','목포시']
orders.loc[orders.CNEE_ADDR_1=='경기도수원시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경기도','수원시']
orders.loc[orders.CNEE_ADDR_1=='경기도시흥시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경기도','시흥시']
orders.loc[orders.CNEE_ADDR_1=='충남금산군',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['충청남도','금산군']
orders.loc[orders.CNEE_ADDR_1=='인천광역시미추홀구',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['인천광역시','미추홀구']
orders.loc[orders.CNEE_ADDR_1=='청주시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['충청북도','청주시']
orders.loc[orders.CNEE_ADDR_1=='경기도수원시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경기도','수원시']
orders.loc[orders.CNEE_ADDR_1=='경기도시흥시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경기도','시흥시']
orders.loc[orders.CNEE_ADDR_1=='충남금산군',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['충청남도','금산군']
orders.loc[orders.CNEE_ADDR_1=='인천광역시미추홀구',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['인천광역시','미추홀구']
orders.loc[orders.CNEE_ADDR_1=='영천시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경상북도','영천시']
orders.loc[orders.CNEE_ADDR_1=='청주시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['충청북도','청주시']
orders.loc[orders.CNEE_ADDR_1=='태종로711번길',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경상북도','경주시']
orders.loc[orders.CNEE_ADDR_1=='道路名稱首爾特別市中區麻將路1街22號（STUDIO',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['서울특별시','중구']
orders.loc[orders.CNEE_ADDR_1=='강서구',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['서울특별시', '강서구']
orders.loc[orders.CNEE_ADDR_1=='마포구',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['서울특별시','마포구']
orders.loc[orders.CNEE_ADDR_1=='영등포구',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['서울특별시','영등포구']

orders.loc[orders.CNEE_ADDR_1=='울특별시','CNEE_ADDR_1'] = '서울특별시'
orders.loc[orders.CNEE_ADDR_1=='강원','CNEE_ADDR_1'] = '강원도'
orders.loc[orders.CNEE_ADDR_1=='서울','CNEE_ADDR_1'] = '서울특별시'
orders.loc[orders.CNEE_ADDR_1=='경북','CNEE_ADDR_1'] = '경상북도'
orders.loc[orders.CNEE_ADDR_1=='대전','CNEE_ADDR_1'] = '대전광역시'
orders.loc[orders.CNEE_ADDR_1=='광주','CNEE_ADDR_1'] = '광주광역시'
orders.loc[orders.CNEE_ADDR_1=='전북','CNEE_ADDR_1'] = '전라북도'
orders.loc[orders.CNEE_ADDR_1=='경남','CNEE_ADDR_1'] = '경상남도'
orders.loc[orders.CNEE_ADDR_1=='대구','CNEE_ADDR_1'] = '대구광역시'
orders.loc[orders.CNEE_ADDR_1=='세종','CNEE_ADDR_1'] = '세종특별자치시'
orders.loc[orders.CNEE_ADDR_1=='\u200b서울시','CNEE_ADDR_1'] = '서울특별시'
orders.loc[orders.CNEE_ADDR_1=='대구시','CNEE_ADDR_1'] = '대구광역시'
orders.loc[orders.CNEE_ADDR_1=='\t경기도','CNEE_ADDR_1'] = '경기도'
orders.loc[orders.CNEE_ADDR_1=='경기','CNEE_ADDR_1'] = '경기도'
orders.loc[orders.CNEE_ADDR_1=='인천','CNEE_ADDR_1'] = '인천광역시'
orders.loc[orders.CNEE_ADDR_1=='전남','CNEE_ADDR_1'] = '전라남도'
orders.loc[orders.CNEE_ADDR_1=='부산','CNEE_ADDR_1'] = '부산광역시'
orders.loc[orders.CNEE_ADDR_1=='충남','CNEE_ADDR_1'] = '충청남도'
orders.loc[orders.CNEE_ADDR_1=='충북','CNEE_ADDR_1'] = '충청북도'
orders.loc[orders.CNEE_ADDR_1=='울산','CNEE_ADDR_1'] = '울산광역시'
orders.loc[orders.CNEE_ADDR_1=='부산시','CNEE_ADDR_1'] = '부산광역시'
orders.loc[orders.CNEE_ADDR_1=='인천시','CNEE_ADDR_1'] = '인천광역시'
orders.loc[orders.CNEE_ADDR_1=='서울시','CNEE_ADDR_1'] = '서울특별시'
orders.loc[orders.CNEE_ADDR_1=='제주','CNEE_ADDR_1'] = '제주특별자치도'
orders.loc[orders.CNEE_ADDR_1=='세종시','CNEE_ADDR_1'] = '세종특별자치시'
orders.loc[orders.CNEE_ADDR_1=='화성시','CNEE_ADDR_1'] = '경기도'
orders.loc[orders.CNEE_ADDR_1=='전남','CNEE_ADDR_1'] = '전라남도'
orders.loc[orders.CNEE_ADDR_1=='울특별시','CNEE_ADDR_1'] = '서울특별시'
orders.loc[orders.CNEE_ADDR_1=='강원','CNEE_ADDR_1'] = '강원도'
orders.loc[orders.CNEE_ADDR_1=='서울','CNEE_ADDR_1'] = '서울특별시'
orders.loc[orders.CNEE_ADDR_1=='경북','CNEE_ADDR_1'] = '경상북도'
orders.loc[orders.CNEE_ADDR_1=='대전','CNEE_ADDR_1'] = '대전광역시'
orders.loc[orders.CNEE_ADDR_1=='광주','CNEE_ADDR_1'] = '광주광역시'
orders.loc[orders.CNEE_ADDR_1=='전북','CNEE_ADDR_1'] = '전라북도'
orders.loc[orders.CNEE_ADDR_1=='경남','CNEE_ADDR_1'] = '경상남도'
orders.loc[orders.CNEE_ADDR_1=='대구','CNEE_ADDR_1'] = '대구광역시'
orders.loc[orders.CNEE_ADDR_1=='세종','CNEE_ADDR_1'] = '세종특별자치시'
orders.loc[orders.CNEE_ADDR_1=='\u200b서울시','CNEE_ADDR_1'] = '서울특별시'
orders.loc[orders.CNEE_ADDR_1=='대구시','CNEE_ADDR_1'] = '대구광역시'
orders.loc[orders.CNEE_ADDR_1=='\t경기도','CNEE_ADDR_1'] = '경기도'
orders.loc[orders.CNEE_ADDR_1=='경기','CNEE_ADDR_1'] = '경기도'
orders.loc[orders.CNEE_ADDR_1=='인천','CNEE_ADDR_1'] = '인천광역시'
orders.loc[orders.CNEE_ADDR_1=='전남','CNEE_ADDR_1'] = '전라남도'
orders.loc[orders.CNEE_ADDR_1=='부산','CNEE_ADDR_1'] = '부산광역시'
orders.loc[orders.CNEE_ADDR_1=='충남','CNEE_ADDR_1'] = '충청남도'
orders.loc[orders.CNEE_ADDR_1=='충북','CNEE_ADDR_1'] = '충청북도'
orders.loc[orders.CNEE_ADDR_1=='울산','CNEE_ADDR_1'] = '울산광역시'
orders.loc[orders.CNEE_ADDR_1=='부산시','CNEE_ADDR_1'] = '부산광역시'
orders.loc[orders.CNEE_ADDR_1=='인천시','CNEE_ADDR_1'] = '인천광역시'
orders.loc[orders.CNEE_ADDR_1=='서울시','CNEE_ADDR_1'] = '서울특별시'
orders.loc[orders.CNEE_ADDR_1=='제주','CNEE_ADDR_1'] = '제주특별자치도'
orders.loc[orders.CNEE_ADDR_1=='세종시','CNEE_ADDR_1'] = '세종특별자치시'
orders.loc[orders.CNEE_ADDR_1=='화성시','CNEE_ADDR_1'] = '경기도'
orders.loc[orders.CNEE_ADDR_1=='전남','CNEE_ADDR_1'] = '전라남도'
orders.loc[orders.CNEE_ADDR_1=='경삼남도','CNEE_ADDR_1'] = '경상남도'
orders.loc[orders.CNEE_ADDR_1=='종시','CNEE_ADDR_1'] = '세종특별자치시'
orders.loc[orders.CNEE_ADDR_1=='청남도','CNEE_ADDR_1'] = '충청남도'

orders.loc[orders.CNEE_ADDR_1=='[16817]',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경기도', '용인시']
orders.loc[orders.CNEE_ADDR_1=='(14782)',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경기도', '부천시']
orders.loc[orders.CNEE_ADDR_1=='(30100)',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['세종특별자치시', '세종로']
orders.loc[orders.CNEE_ADDR_1=='(22738)',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['인천광역시', '서구']
orders.loc[orders.CNEE_ADDR_1=='(53298)',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경상남도', '거제시']
orders.loc[orders.CNEE_ADDR_1=='(13279)',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경기도', '성남시']
orders.loc[orders.CNEE_ADDR_1=='(13499)',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경기도', '성남시']
orders.loc[orders.CNEE_ADDR_1=='(12730)', ['CNEE_ADDR_1','CNEE_ADDR_2']] = ['인천광역시','서구']
orders.loc[orders.CNEE_ADDR_1=='인천광역시\xa0서구\xa0정서진로\xa047',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['인천광역시','서구']
orders.loc[orders.CNEE_ADDR_1=='평택시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경기도','평택시']
orders.loc[orders.CNEE_ADDR_1=='곤지암',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경기도','광주시']
orders.loc[orders.CNEE_ADDR_1=='상주시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경상북도','상주시']

orders.loc[orders.CNEE_ADDR_2=='충남공주시',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['충청남도','공주시']
orders.loc[orders.CNEE_ADDR_2=='경상남도',['CNEE_ADDR_1','CNEE_ADDR_2']] = ['경상남도',np.nan]

In [ ]:
orders.to_csv('orders.csv', index=False)

In [ ]:
orders = pd.read_csv('orders.csv')
deliveries = pd.read_csv('deliveries.csv')

In [ ]:
orders.BKG_DATE = pd.to_datetime(orders.BKG_DATE, format='%Y-%m-%d')
orders.INS_DATE = pd.to_datetime(orders.INS_DATE, format='%Y-%m-%d')

## EDA

### Orders

#### column 설명
- CORP_ID: 창고코드(KX007: 곤지암, GP001: 군포)
- REF_ORD_NO: 고객 주문번호, 해당 테이블 key값, 네이버 채번
- BKG_NO: CJ대한통운 주문번호
- BKG_TYP: 주문유형(7: B2C출고, 8: 정상반출, 9: 불량반출) 
- BKG_DATE: 주문날짜
- BKG_TIME: 주문시간
- SHPR_CD: 고객사 코드
- INV_AMT: 주문금액
- ITEM_SEQ: 품목순번
- ITEM_CD: 품목코드
- BRAND_NM: 브랜드
- ITEM_QTY: 품목수량
- ITEM_AMT: 품목금액
- IF_YN: 수신여부
- ORDER_CRT_DATETIME: 주문생성시간
- DLV_DV: 택배구분
- REF_ITEM_SEQ: 상품주문번호
- ORDER_IDX: 중개업체 주문번호
- ORDER_YN: 접수여부
- DLVPREARRBRANCD: 배달예정점소코드
- DLVPREARREMPNICKNM: 배달예정사원분류코드
- DLVCLSFCD: 배달터미널코드
- DLVSUBCLSFCD: 배달터미널 소분류코드
- INS_ID: 입력자ID
- INS_DATE: 입력일자(인터페이스시간)
- INS_TIME: 입력시간(인터페이스시간)
- POST_ZONE: 권역구분
- SPLIT_EXEC_YN: 배송처별 주문분할여부
- SHPR_ADDR_1: 송화인 주소1(시도)
- SHPR_ADDR_2: 송화인 주소2(시군구)
- CNEE_ADDR_1: 수화인 주소1(시도)
- CNEE_ADDR_2: 수화인 주소2(시군구)

In [ ]:
orders.head()

In [ ]:
msno.bar(orders)
plt.show()

In [ ]:
# 창고코드
orders.CORP_ID.value_counts()

In [ ]:
# 주문유형
orders.BKG_TYP.value_counts()

In [ ]:
# 고객사 번호
print(f'nunique: {orders.SHPR_CD.nunique()}')
print(orders.SHPR_CD.value_counts())

In [ ]:
# 품목순번
orders.ITEM_SEQ.nunique()

In [ ]:
# 품목코드
orders.ITEM_CD.nunique()

In [ ]:
# 브랜드명
orders.BRAND_NM.value_counts()

In [ ]:
# 수신여부
orders.IF_YN.value_counts()

In [ ]:
# 택배 구분
orders.DLV_DV.value_counts()

In [ ]:
# 접수여부
orders.ORDER_YN.value_counts()

In [ ]:
# 배달예정점소코드
orders.DLVPREARRBRANCD.nunique()

In [ ]:
# 배달예정사원분류코드
orders.DLVPREARREMPNICKNM.nunique()

In [ ]:
# 배달터미널코드
orders.DLVCLSFCD.nunique()

In [ ]:
# 배달터미널소분류코드
orders.DLVSUBCLSFCD.nunique()

In [ ]:
orders.DLVSUBCLSFCD.value_counts()

In [ ]:
# 입력자 ID
print(f'nunique: {orders.INS_ID.nunique()}')
orders.INS_ID.value_counts()

In [ ]:
# 권역구분
orders.POST_ZONE.nunique()

In [ ]:
orders.POST_ZONE.value_counts()

In [ ]:
# 배송처별 주문분할여부
orders.SPLIT_EXEC_YN.value_counts()

In [ ]:
orders.SHPR_ADDR_1.value_counts()

In [ ]:
orders.SHPR_ADDR_2.value_counts()

In [ ]:
orders.CNEE_ADDR_1.value_counts()

In [ ]:
orders.CNEE_ADDR_2.value_counts()

### Delivery

#### column 설명
- 구분명: 배달/집하
- 집화일자: 집화(화물이나 상품 따위가 한곳에 모여들거나 모이게 함)
- 집배일자: 집배(우편물이나 화물 따위를 모아서 주소지로 배달하는 따위를 이른다.)
- 운임명: 선불/신용/착불
- 수량: box
- 운임: 원
- 집화여부: 여부(Y/N)
- 집배시간: 
- 배달일자: 배달/집하 완료 일자
- 장비구분: 화면/스마트폰
- 품목: 배송상품 품목 분류
- SM명: 택배기사 성명
- 받는분 주소

In [ ]:
msno.bar(deliveries)
plt.show()

In [ ]:
deliveries.isna().sum()[deliveries.isna().sum()!=0]

In [ ]:
deliveries.head()

In [ ]:
deliveries.구분명.value_counts()

In [ ]:
deliveries.운임명.value_counts()

In [ ]:
deliveries.집화여부.value_counts()

In [ ]:
deliveries.장비구분.value_counts()

In [ ]:
deliveries.품목.value_counts()

In [ ]:
deliveries.SM명.nunique()